# Predicting credit default with XGBoost
_**Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem **_

_NOTE: This notebook was created and tested using the `Python 3 (Data Science)` kernel._

---

---

## Contents

1. [Background](#Background)
1. [Preparation](#Preparation)
1. [Data](#Data)
    1. [Exploration](#Exploration)
    1. [Transformation](#Transformation)
1. [Training](#Training)
1. [Hosting](#Hosting)
1. [Evaluation](#Evaluation)
1. [Extensions](#Extensions)

---

## Background

We will be using a dataset from the UCI Machine Learning Repository

This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005.

The original dataset can be found here at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients



## Install additional libraries

[Pandas profiling](https://github.com/pandas-profiling/pandas-profiling) works great for smaller datasets. Pandas profiling does not come preinstalled on the Data Science Kernel, but we can install it with pip.

In [ ]:
!pip install pandas-profiling[notebook] --quiet
!pip install sagemaker-experiments --quiet

## Preparation

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
import boto3
import re
import sagemaker # Amazon SageMaker's Python SDK provides many helper functions

session = sagemaker.Session()

bucket = session.default_bucket()
prefix = 'sagemaker/credit-xgboost'
 
# Define IAM role
role = sagemaker.get_execution_role()

In [ ]:
(bucket,prefix,role)

Now let's bring in the Python libraries that we'll use throughout the analysis

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker.deserializers                    # Converts strings for HTTP POST requests on inference

---

## Data

The dataset should already be available in the repository under the dataset folder. Now lets read this into a Pandas data frame and take a look.


In [ ]:
data = pd.read_csv('./dataset/UCI_Credit_Card.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

Let's talk about the data.  At a high level, we can see:

* We have 30000 user and their credits records
* 25 features per record
* The features are numeric
* The data is not sorted

_**Specifics on each of the features:**_

*Features:*
* `ID`: ID of user
* `LIMIT_BAL`: Amount of given credit in NT dollars (includes individual and family/supplementary credit
* `SEX`: Gender (1=male, 2=female)
* `EDUCATION`: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
* `Marriage`: Marital status (1=married, 2=single, 3=other)
* `AGE`: Age in years
* `PAY_0`: Repayment status for this month. (-2=No consumption, -1=paid duly, 0=use of revolving credit, 1=payment delayed 1 month, 2=payment delayed 2 months, [...], 9=payment delayed 9 months or more)
* `PAY_2`: Repayment status for last month. (same as above)
* `PAY_3`: Repayment status for 2 months ago. (same as above)
* `PAY_4`: Repayment status for 3 months ago. (same as above)
* `PAY_5`: Repayment status for 4 months ago. (same as above)
* `PAY_6`: Repayment status for 5 months ago. (same as above)
* `BILL_AMT1`: Amount of bill this month.
* `BILL_AMT2`: Amount of bill last month.
* `BILL_AMT3`: Amount of bill 2 months ago.
* `BILL_AMT4`: Amount of bill 3 months ago.
* `BILL_AMT5`: Amount of bill 4 months ago.
* `BILL_AMT6`: Amount of bill 5 months ago.
* `PAY_AMT1`: Amount of payment made this month.
* `PAY_AMT2`: Amount of payment made last month.
* `PAY_AMT3`: Amount of payment made 2 months ago.
* `PAY_AMT4`: Amount of payment made 3 months ago.
* `PAY_AMT5`: Amount of payment made 4 months ago.
* `PAY_AMT6`: Amount of payment made 5 months ago.

*Target variable:*
* `default.payment.next.month`: Default payment next month (1=yes, 0=no).



### Data Exploration
Let's start exploring the data.  First, let's understand how the features are distributed.

In [ ]:
from pandas_profiling import ProfileReport

profile_data = data
profile = ProfileReport(profile_data, title="Credit card default data set report", minimal=True)
profile

In [ ]:
data.info()

### First Training (Local)

Let's first train a simple decision tree model on our data. Our target variable is 'default.payment.next.month'. We will start with a tree depth of 10. We split our data into a training dataset consisting of 80% of the samples, and a test dataset with 20% of the samples. We also drop the ID column as it's a simple record identifier with no predictive value

In [ ]:
#importing libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(['ID','default.payment.next.month'], axis=1), data.filter(like='default.payment.next.month'), test_size=0.20, random_state=42)

classifier = DecisionTreeClassifier(max_depth=10, random_state=14) 
classifier.fit(X_train, y_train)

predictions_test = classifier.predict(X_test)
predictions_test_prob = classifier.predict_proba(X_test)

precision = precision_score(y_test, predictions_test)
recall = recall_score(y_test, predictions_test)
accuracy = accuracy_score(y_true = y_test, y_pred = predictions_test)
f1 = f1_score(y_true = y_test, y_pred = predictions_test)
auc_score = roc_auc_score(y_test, predictions_test_prob[:,1])

print('Precision (tp / (tp + fp)): ', np.round(precision, 3))
print('Recall (tp/(tp + fn))): ', np.round(recall, 3))
print('Accuracy: ', np.round(accuracy, 3))
print('F1 score: ', np.round(f1, 3))
print('AUC score: ', np.round(auc_score, 3))

print('\nConfusion matrix for 0.5:')
pd.crosstab(index=y_test.iloc[:,0].values, columns=predictions_test, rownames=['defaulted on payment'], colnames=['predicted value']) 


Although we have good accuracy, our F1 score is quite low, due to the imbalance in classes - there are many more cases of not defaulting on payment so the model is biased. We have also more false negatives compared to false positives, and as such our precision is better than our recall for this model at a decision boundary of 0.5 probability.

For binary classification, scikit-learn will by default make the assumption that 1 is the positive class (in this case the target 'thing' we are predicting, which is defaulting on credit), and 0 is the negative class. Our average precision is shown above to be around 36% - which is the amount of true positives among all inferred positives at different probabilistic decision thresholds 0 < thresh < 1. Precision is a good measure to determine, when the costs of False Positive is high.

We can also see that the accuracy (how many of the total samples were correctly predicted) with our test dataset is around 81% and with our train dataset is around 84% meaning we are not overfitting to the training data.

What would happen if we would use a tree depth of 100?

In [ ]:
classifier = DecisionTreeClassifier(max_depth=100, random_state=14) 
classifier.fit(X_train, y_train)

# test data set accuracy
predictions = classifier.predict(X_test)
print("Test set accuracy: {}".format(accuracy_score(y_true = y_test, y_pred = predictions)))

# train data set accuracy (to measure overfitting)
predictions = classifier.predict(X_train)
print("Train set accuracy: {}".format(accuracy_score(y_true = y_train, y_pred = predictions)))

Clearly now there is overfitting to the training data going on, with our model giving much better results on the data with which we trained than with data not seen before (test data)

### Data Feature Engineering

Let's try to do some feature engineering and improve on the previous model accuracy.

**Drop meaningless features**

ID is used as a record identified and is not useful from a model training or prediction point of view

In [ ]:
data = data.drop(['ID'], axis=1)


Education of type 5, 6, and 0 are unknown, so let's combine them into 4 ('Others').

In [ ]:
# Education 5,6,and 0 are unknown, so set to 4 'Others'
print("Before:")
print(data.EDUCATION.value_counts())
print()

data.loc[data['EDUCATION'] > 3 , 'EDUCATION'] = 4
data.loc[data['EDUCATION'] == 0 , 'EDUCATION'] = 4

print("After:")
print(data.EDUCATION.value_counts())
print()

# The same for Marriage, unknown value 0 can be set to 3 'Others'
print("Before:")
print(data.MARRIAGE.value_counts())
print()

data.loc[data['MARRIAGE'] == 0 , 'MARRIAGE'] = 3

print("After:")
print(data.MARRIAGE.value_counts())

### MinMax scaler to numerical features
Scale the numerical features using a MinMax scaler


In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

columns_to_scale = ['AGE', 'LIMIT_BAL', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

data[columns_to_scale] = min_max_scaler.fit_transform(data[columns_to_scale])

data


#### One-hot encode categorical values

The Sex, Education and Marriage features are categorical with no inherent meaning to the ordering, which is why we one-hot encode them.



In [ ]:
columns_to_onehot_encode = ['SEX', 'EDUCATION', 'MARRIAGE']

model_data = pd.get_dummies(data, prefix=columns_to_onehot_encode, columns=columns_to_onehot_encode)

model_data

#### Move target column to front. 
SageMaker assumes first column is the target column.

In [ ]:
model_data = model_data[ ['default.payment.next.month'] + [ col for col in model_data.columns if col != 'default.payment.next.month' ] ]

model_data

### Split data

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10% 

Upload train and validation data to S3


In [ ]:
train_data.to_csv('train.csv', index=False, header=False)
validation_data.to_csv('validation.csv', index=False, header=False)

# Upload the dataset to our S3 bucket
train_uri = session.upload_data(path='train.csv', key_prefix=prefix+'/train')
validation_uri = session.upload_data(path='validation.csv', key_prefix=prefix+'/validation')

print(train_uri)
print(validation_uri)

### SageMaker Processing Jobs

We did the previous data preparation steps on our notebook - for larger scale handling of data, we could use SageMaker Processing and Processing jobs to do the data preparation / feature engineering using a managed cluster. In our case, we used Scikit Learn, and SageMaker Processing supports it

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0",
    role=role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
)

from sagemaker.processing import ProcessingInput, ProcessingOutput

# We now upload the raw dataset
cc_data_uri = session.upload_data(path='dataset/UCI_Credit_Card.csv', key_prefix=prefix+'/data')
print(cc_data_uri)

sklearn_processor.run(
    code="./preprocessing.py",
    inputs=[
        ProcessingInput(source=cc_data_uri, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output", destination="s3://{}/{}/processed".format(bucket, prefix))
    ]
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

Now we have our engineered data on S3, and we can download to compare with the same data we created in the notebook

In [ ]:
session.download_data("processing", bucket, key_prefix="{}/processed".format(prefix))

### SageMaker Training Jobs

We will now train on our data using the SageMaker XGBoost Algorithm.

First, we fetch the xgboost container

In [ ]:
container = sagemaker.image_uris.retrieve('xgboost', boto3.Session().region_name, version='latest')

#### Create input channels
SageMaker work with the concept of channels. Here, we create a training and validation input channles to be used when training


In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation'.format(bucket, prefix), content_type='csv')


#### Train the model
To train the model, we
* initiate a SageMaker session,
* create an estimator object and specify training configuration,
* set the hyperparameters,
* call the fit method, passing in the training and validation channels

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    use_spot_instances=True,
                                    max_run=3600,
                                    max_wait=3600,
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=25)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

### Deploy model

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

Specify how to serialize incoming data

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer() 

Throw some test data at the endpoint

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['default.payment.next.month'], axis=1).to_numpy())

Compute some metrics to evaluate model

In [ ]:
from sklearn.metrics import average_precision_score, accuracy_score, f1_score

y_true = test_data['default.payment.next.month']

average_precision = average_precision_score(y_true, np.round(predictions))
accuracy = accuracy_score(y_true, np.round(predictions))
f1 = f1_score(y_true = y_true, y_pred = np.round(predictions))

print('Precision: ', np.round(average_precision, 3))
print('Accuracy: ', np.round(accuracy, 3))
print('F1 score: ', np.round(f1, 3))


print('\nConfusion matrix:')
pd.crosstab(index=test_data['default.payment.next.month'], columns=np.round(predictions), rownames=['defaulted on payment'], colnames=['predictions'])

Create ROC graph

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import roc_auc_score

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(test_data['default.payment.next.month'], predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

#### Clean up!
When you're done, don't forget to tear down the endpoint.

In [ ]:
xgb_predictor.delete_endpoint()

### Hyper parameter tuning
There are many ways to tune the model hyper parameters. One way is to use SageMaker Automatic Hyperparameter Tuning.

With Automatic Hyperparameter Tuning, we create a HyperparameterTuner and pass in the estimator we created before, define ranges for the parameters we want to tune, define which metric to optimise for and add some training configuration.

We then call the fit method and pass in the training and validation channels.

Once the tuning is complete, we deploy the tuner and the model with the best result in the tuning phase will automatically be chosen.



In [ ]:
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter

# Configure HyperparameterTuner
my_tuner = HyperparameterTuner(estimator=xgb,  # previously-configured Estimator object
                               objective_metric_name='validation:auc',
                               hyperparameter_ranges={
                                   'eta': ContinuousParameter(0, 0.5), 
                                   "alpha": ContinuousParameter(0, 1000), 
                                   "min_child_weight": ContinuousParameter(1, 120), 
                                   "max_depth": IntegerParameter(1, 10),
                                   "num_round": IntegerParameter(1, 2000),
                                   "subsample": ContinuousParameter(0.5, 1)
                               },
                               max_jobs=10,
                               max_parallel_jobs=10)

# Start hyperparameter tuning job
my_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

# Deploy best model
print('Deploying the model ')
my_predictor = my_tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')



Let's move all our training jobs into a SageMaker *Experiment* so we can track their performance and rank them visually.

In [ ]:
from smexperiments.experiment import Experiment
import botocore.exceptions
import uuid

sm = boto3.Session().client('sagemaker')
trial_name = 'hyper-parameter-tuning'

# Let's create an Experiment with a name - Credit-default for example
try:
    my_experiment = Experiment.create(experiment_name='credit-default')
except botocore.exceptions.ClientError as e:
    print("Experiment already created...")
    pass
# And a trial within that Experiment
try:
    my_trial = my_experiment.create_trial(trial_name=trial_name)
except botocore.exceptions.ClientError:
    print("Trial already created...")
    pass

# Now let's move our training jobs into the trial, as trial components.
for trial_component in sm.list_trial_components(MaxResults=100)['TrialComponentSummaries']:
    if trial_component['TrialComponentSource']['SourceType'] == 'SageMakerTrainingJob':
        sm.associate_trial_component(TrialComponentName=trial_component['TrialComponentName'], TrialName=trial_name)

From the SageMaker Studio 'Components and registries' on the sidebar, if you select 'Experiments and trials' and then navigate to the Experiment and Trial we just created, you can then select the multiple trial components (our hpt training jobs), sort them, deploy a model from the most suitable one, and create charts.

#### Test new model
As before, we inform the predictor how to serialize the incoming data, then we run a set of predictions

In [ ]:
my_predictor.serializer = sagemaker.serializers.CSVSerializer() 

# Make a prediction against the SageMaker endpoint
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, my_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['default.payment.next.month'], axis=1).to_numpy())

#### Model evaluation
Let's evaluate the performance of the new model

In [ ]:
y_true = test_data['default.payment.next.month']

average_precision = average_precision_score(y_true, np.round(predictions))
accuracy = accuracy_score(y_true, np.round(predictions))
f1 = f1_score(y_true = y_true, y_pred = np.round(predictions))

print('Precision: ', np.round(average_precision, 3))
print('Accuracy: ', np.round(accuracy, 3))
print('F1 score: ', np.round(f1, 3))


print('\nConfusion matrix:')
pd.crosstab(index=test_data['default.payment.next.month'], columns=np.round(predictions), rownames=['defaulted on payment'], colnames=['predictions'])

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import roc_auc_score

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(test_data['default.payment.next.month'], predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

#### Clean up!
When you're done, don't forget to tear down the endpoint.

In [ ]:
my_predictor.delete_endpoint()